In [1]:
import random
import sqlite3 as sqlite
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

app = Flask(__name__)

line_bot_api = LineBotApi('ai1iuZRwsnNXq0c351M2hRHF/WCgp1d8vVRo758C4Pf4zFdUlc9niSEU/RaSnMm6c236XIItMn06tjMEs8G/ZIE7NrQnl8brQxWlwC6erjgTiTq2DbXn5PEtrnZwehMN8CP4uRjdUKuBIO7KHJf+rgdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('1ca71e0fc847d319ce3e2b8497de7baf')



In [ ]:
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
        
    if event.message.text=='桌遊啦':
        conn = sqlite.connect('class.db')
        c = conn.cursor()
        c.execute('CREATE TABLE '+event.source.group_id+' (menber text UNIQUE,acter text UNIQUE)')
        conn.commit()
        conn.close()
        print('以創建'+event.source.group_id)
    elif event.message.text=='+1':
        conn = sqlite.connect('class.db')
        c = conn.cursor()
        c.execute("INSERT INTO "+event.source.group_id+"(menber) VALUES ('"+event.source.user_id+"')")
        conn.commit()
        conn.close()
        line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='已加入~'))
    elif event.message.text=='開始':
        conn = sqlite.connect('class.db')  
        c = conn.cursor()  
        menbers=list(c.execute("SELECT menber FROM "+event.source.group_id))
        conn.close()
        
        select=random.randint(0,len(menbers)-1)
        spy=menbers[select]
        menbers.remove(spy)
        spy=spy[0]
        print(event.source.group_id)
        line_bot_api.push_message(spy, TextSendMessage(text='你是spy'))
        
        user_id="'%s'"%event.source.user_id
        conn = sqlite.connect('class.db')
        c = conn.cursor()
        c.execute("UPDATE %s SET acter = 'spy' WHERE menber = %s"%(event.source.group_id,user_id))
        conn.commit()
        conn.close()
        
        print(menbers)
        for item in menbers:
            line_bot_api.push_message(item[0], TextSendMessage(text='你是警察'))
            
            user_id="'%s'"%item[0]
            conn = sqlite.connect('class.db')
            c = conn.cursor()
            c.execute("UPDATE %s SET acter = 'cup' WHERE menber = %s"%(event.source.group_id,user_id))
            conn.commit()
            conn.close()
        

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Aug/2018 01:32:27] "POST /callback HTTP/1.1" 200 -


以創建C12057029888574587e0746aff6d6e9de


127.0.0.1 - - [06/Aug/2018 01:32:39] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2018 01:32:40] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2018 01:32:52] "POST /callback HTTP/1.1" 200 -


C12057029888574587e0746aff6d6e9de
[('U741a582b1679a0feb05d7ac2d6fe2b14',)]


127.0.0.1 - - [06/Aug/2018 01:33:00] "POST /callback HTTP/1.1" 200 -


In [47]:
menbers=['ss','dd']

In [48]:
menbers.remove('ss')

In [49]:
menbers

['dd']